In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Aug 28 15:49:43 2018

@author: charlie
"""

# Map idea based on: http://ukdataexplorer.com/renewables/
# And further details based on: http://www.renewables-map.co.uk/subscriber/capacityfactor.asp
#
# Map data from: https://www.gov.uk/government/publications/renewable-energy-planning-database-monthly-extract
# ROC data from: https://www.renewablesandchp.ofgem.gov.uk/Public/ReportManager.aspx?ReportVisibility=1&ReportCategory=0 

# import libraries

import pandas as pd # for dataframes
import numpy as np  # for arrays
#import geoviews as gv   # not needed?
#from cartopy import crs # not needed?
from pyproj import Proj, transform # for coordinate transformation
from bokeh.tile_providers import STAMEN_TERRAIN          # for map tile
from bokeh.plotting import figure, output_notebook, show # for plotting map
from bokeh.models import HoverTool
from bokeh.plotting import figure, output_file, show, ColumnDataSource
output_notebook() # to plot to notebook


# import date

# data from: https://www.gov.uk/government/publications/renewable-energy-planning-database-monthly-extract
file_name = r'renewable-energy-planning-database-december-2019.csv'
renewablesData = pd.read_csv(file_name,encoding='ISO-8859-1',skiprows=1)

renewablesData.head()

renewablesData.columns

# clean data set

# only keep sites which are operational
renewablesData = renewablesData[pd.notnull(renewablesData['Operational'])]

# shorten the data set to useful parameters
coordinates = renewablesData[['Site Name','X-coordinate','Y-coordinate','Installed Capacity (MWelec)','Technology Type','Operational']]
coordinates = coordinates[pd.notnull(coordinates['X-coordinate'])]

# set index
#coordinates = coordinates.set_index('Site Name')

coordinates
coordinates.index

# convert coordinates to values from strings
coordinates['X-coordinate'] = pd.to_numeric(coordinates['X-coordinate'].str.replace(',',''),errors='coerce')
coordinates['Y-coordinate'] = pd.to_numeric(coordinates['Y-coordinate'].str.replace(',',''),errors='coerce')
coordinates['Installed Capacity (MWelec)'] = pd.to_numeric(coordinates['Installed Capacity (MWelec)'].str.replace(',',''),errors='coerce')
#print(coordinates['Installed Capacity (MWelec)'])

# convert OSBG coordinates (Ordance Surver British National Grid)

# epsg coordinate system codes for transformations https://epsg.io/27700
#epsg:27700 is for OSBG
#epsg:3857 is for OpenMaps
#epsg:4326 is for longitude and latitude

# transformation done using Proj, http://proj4.org/usage/resource_files.html 

inProj = Proj(init='epsg:27700')
outProj = Proj(init='epsg:3857')

coordXOS = coordinates['X-coordinate'].values
coordYOS = coordinates['Y-coordinate'].values

coordX,coordY = transform(inProj,outProj,coordXOS,coordYOS)
coordinates['X-coordinateOM'],coordinates['Y-coordinateOM'] = transform(inProj,outProj,coordXOS,coordYOS)

inProj = Proj(init='epsg:27700')
outProj = Proj(init='epsg:4326')

coordinates['Longitude'],coordinates['Latitude'] = transform(inProj,outProj,coordXOS,coordYOS)


# define map limits based on coordinates
minX = min(coordX)
maxX = max(coordX)
minY = min(coordY)
maxY = max(coordY)
# print(minX,maxX,minY,maxY)


UK = x_range, y_range = ((minX,maxX), (minY,maxY))

plot_width  = int(750)
plot_height = int(plot_width//1.2)


def base_plot(tools='pan,zoom_in,zoom_out,wheel_zoom,box_zoom,reset',plot_width=plot_width, plot_height=plot_height, **plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p
    
options = dict(line_color=None, fill_color='blue', size=5)


# coordinates.columns


from matplotlib.colors import LinearSegmentedColormap

#print(coordinates['Technology Type'].unique())

cdict = {'Biomass (co-firing)': 'brown',
         'EfW Incineration': 'coral', #cyan
         'Biomass (dedicated)':  'beige',
         'Advanced Conversion Technologies': 'blueviolet',
         'Anaerobic Digestion': 'chocolate',
         'Large Hydro': 'blue',
         'Small Hydro': 'cornflowerblue',
         'Landfill Gas': 'chartreuse',
         'Solar Photovoltaics': 'yellow',
         'Sewage Sludge Digestion': 'darkolivegreen',
         'Tidal Barrage and Tidal Stream': 'turquoise',
         'Shoreline Wave': 'indigo',
         'Wind Offshore': 'green',
         'Wind Onshore': 'forestgreen',
         'Hot Dry Rocks (HDR)': 'pink',
                                  }

cmap = LinearSegmentedColormap('tech', cdict)

coordinates['colours'] = coordinates['Technology Type'].map(cdict)

#print(coordinates[['Technology Type','colours']])


from bokeh.palettes import Category20, inferno, plasma, viridis

p = base_plot()

p.add_tile(STAMEN_TERRAIN)

technologyTypes = coordinates['Technology Type'].unique()
#print(technologyTypes)

#colours = Category20[len(technologyTypes)]
#colours = inferno(len(technologyTypes))
colours = viridis(len(technologyTypes))
for tech,colour in zip(technologyTypes,colours):
    # print(tech)
    
    techData = coordinates.loc[coordinates['Technology Type'] == tech]
    #print(techData)
    
    source = ColumnDataSource(data=dict(
        x=techData['X-coordinateOM'],
        y=techData['Y-coordinateOM'],
        #radius=coordinates['Installed Capacity (MWelec)'].values*10,
        radius=2*techData['Installed Capacity (MWelec)'].values**0.5,
        #radius=np.log(coordinates['Installed Capacity (MWelec)'].values)*10,
        #pixels=coordinates['Installed Capacity (MWelec)'].values/10,
        pixels=techData['Installed Capacity (MWelec)'].values**0.5,
        #pixels=np.log(coordinates['Installed Capacity (MWelec)'].values)*10,
        sizeMW=techData['Installed Capacity (MWelec)'].values,
        technology=techData['Technology Type'],
        longitude=techData['Longitude'].values,
        latitude=techData['Latitude'].values,
        operational=techData['Operational'],
        siteName=techData['Site Name'],
        #colours=techData['colours'],

    ))
    
    p.scatter('x', 'y',
          size='radius', 
          alpha=0.9, 
          #fill_color='colours', hover_fill_color="colours", 
          fill_color=colour, 
          hover_fill_color=colour, 
          line_color="black",
          hover_line_color="white",
          muted_alpha=0.1,
          fill_alpha=0.3, 
          hover_alpha=0.5,
          line_width=1, 
          legend_label=tech,
          source=source)


#coordinates['Longitude'].values


p.legend.location = "top_left"
p.legend.click_policy="hide"


# add custom hover tool
hover = HoverTool(tooltips=[
    #("index", "$index"),
    ("Site name","@siteName"),
    #("(x,y)", "($x, $y)"),
    ("Technology", "@technology"),
    ("Size MW", "@sizeMW"),
    #("Long,Lat","@longitude,@latitude"),
    ("Operational","@operational"),
    
    ]
    #renderers=cr #    p.add_glyph()
)

p.add_tools(hover)

show(p)






Loading BokehJS ...